# Modèles avancés (20%) 

In [54]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [55]:
load_dotenv(r"C:\Users\Prevot\Documents\Montreal\Cours\ScienceDesDonnees\Projet\A02-projet\notebooks\.env")

True

In [56]:
comet_api_key = os.environ.get('COMET_API_KEY')

In [66]:
# Importer comet_ml en haut de votre fichier, avant sklearn !
from comet_ml import Experiment

# Créer une expérience avec votre clé api
exp = Experiment(
    api_key=os.environ.get(comet_api_key),#ne pas coder en dur!
    project_name='milestone2',
    workspace='ift6758-a02',
)
# ... Faire de la science des données ...
#exp.log_metrics({"auc": 12, "acc": 0.8, "loss": 0.65})


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, xgboost.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/ift6758-a02/milestone2/238c1170b0f241039a1c7880f3836226
COMET INFO:   Metrics:
COMET INFO:     acc  : 0.8
COMET INFO:     auc  : 12
COMET INFO:     loss : 0.65
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, xgboost.
COMET 

### Loading data

In [58]:
data_path = os.environ.get('DATA_DIR')

In [59]:
train_data_path = data_path+"\\nhl_train_data.csv"
test_data_path = data_path+"\\nhl_test_data.csv"

In [60]:
train_data=pd.read_csv(train_data_path)
test_data=pd.read_csv(test_data_path)

In [61]:
train_data.sample(3)
train_data.shape

(305797, 9)

In [62]:
test_data.sample(3)
test_data.shape

(52408, 9)

### Modèle XGBoost

In [67]:
# on garde que les caractéristique qu'on veut étudier ici
X_train_select = train_data[['distanceToNet', 'relativeAngleToNet']]
y_train_select = train_data['isGoal']

# split l'ensemble de données en ensembles d'entraînement et de validation
X_train, X_valid, y_train, y_valid = train_test_split(X_train_select, y_train_select, test_size=0.2, random_state=42)

# Initialisation du modèle XGBoost
model = xgb.XGBClassifier()

# entraîenement
model.fit(X_train, y_train)

# prédictions sur l'ensemble de validation
y_pred = model.predict(X_valid)

# accuracy
accuracy = accuracy_score(y_valid, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.9063603662524525
